In [1]:
import os
import sys
import torch
import random

sys.path.append('../../shared_libraries')
from trainer import Trainer
from config import get_config
from utils import prepare_dirs, save_config, load_config
from data_loader import get_train_valid_loader, get_test_loader

root_dir = os.getcwd()

# Load configuration

In [2]:
config, unparsed = get_config()

config.num_train = 45000
config.batch_size = 8
config.epochs = 100
config.num_model = 2

# ensure directories are setup
prepare_dirs(config)

config.print_values()

valid_trials : 320
test_trials : 400
way : 20
num_train : 45000
batch_size : 8
num_workers : 1
shuffle : True
augment : True
is_train : True
epochs : 100
init_momentum : 0.5
lr_patience : 1
train_patience : 20
flush : False
num_model : 2
use_gpu : True
best : True
random_seed : 1
data_dir : ./data/changed/
ckpt_dir : ./ckpt/
logs_dir : ./logs/
resume : False


# Configure training

In [3]:
# create Omniglot data loaders
torch.manual_seed(config.random_seed)
kwargs = {}
if config.use_gpu:
    torch.cuda.manual_seed(config.random_seed)
    kwargs = {'num_workers': 1, 'pin_memory': True}
if config.is_train:
    data_loader = get_train_valid_loader(
        config.data_dir, config.batch_size,
        config.num_train, config.augment,
        config.way, config.valid_trials,
        config.shuffle, config.random_seed,
        **kwargs
    )
else:
    data_loader = get_test_loader(
        config.data_dir, config.way,
        config.test_trials, config.random_seed,
        **kwargs
    )

# sample 3 layer wise hyperparams if first time training
if config.is_train and not config.resume:
    print("[*] Sampling layer hyperparameters.")

    layer_hyperparams = {
        'layer_init_lrs': [],
        'layer_end_momentums': [],
        'layer_l2_regs': []
    }
    for i in range(6):
        # sample
        lr = random.uniform(1e-4, 1e-1)
        mom = random.uniform(0, 1)
        reg = random.uniform(0, 0.1)

        # store
        layer_hyperparams['layer_init_lrs'].append(lr)
        layer_hyperparams['layer_end_momentums'].append(mom)
        layer_hyperparams['layer_l2_regs'].append(reg)
    try:
        save_config(config, layer_hyperparams, override=True)
    except ValueError:
        print(
            "[!] Samples already exist. Either change the model number,",
            "or delete the json file and rerun.",
            sep=' '
        )
        raise
# else load it from config file
else:
    try:
        layer_hyperparams = load_config(config)
    except FileNotFoundError:
        print("[!] No previously saved config. Set resume to False.")
        raise

[*] Sampling layer hyperparameters.
[*] Model Checkpoint Dir: ./ckpt/exp_2
[*] Param Path: ./ckpt/exp_2\params.json


# Train with data augmentation

In [4]:
trainer = Trainer(config, data_loader, layer_hyperparams)

trainer.train()

C:\Users\bruno\Desktop\siamese-neural-networks\model.py:46: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(m.weight, 0, 1e-2)
C:\Users\bruno\Desktop\siamese-neural-networks\model.py:47: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(m.bias, 0.5, 1e-2)
C:\Users\bruno\Desktop\siamese-neural-networks\model.py:49: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(m.weight, 0, 2e-1)
C:\Users\bruno\Desktop\siamese-neural-networks\model.py:50: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(m.bias, 0, 1e-2)
  0%|          | 0/45000 [00:00<?, ?it/s]

[*] Number of model parameters: 1,256,657

[*] Train on 45000 sample pairs, validate on 320 trials

Epoch: 1/100


C:\Users\bruno\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
0.0s - loss: 0.495: 100%|██████████| 45000/45000 [04:59<00:00, 150.05it/s]
C:\Users\bruno\Desktop\siamese-neural-networks\trainer.py:249: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)
  0%|          | 0/45000 [00:00<?, ?it/s]

train loss: 0.639 - val acc: 21.250 [*]

Epoch: 2/100


  0%|          | 0/45000 [00:00<?, ?it/s]

...

train loss: 0.095 - val acc: 90.000

Epoch: 93/100


  0%|          | 0/45000 [00:00<?, ?it/s]

train loss: 0.095 - val acc: 89.375

Epoch: 94/100


0.0s - loss: 0.111: 100%|██████████| 45000/45000 [07:22<00:00, 101.78it/s]


train loss: 0.095 - val acc: 86.250
[!] No improvement in a while, stopping training.


# Train with data augmentation and Batch Normalization

In [8]:
config, unparsed = get_config()

config.num_train = 45000
config.batch_size = 64
config.epochs = 100
config.num_model = 4
config.augment = True
config.use_gpu = False
config.num_workers = 6
config.use_batch_norm = True
config.data_dir = '../../01_data_preparation/omniglot/data/changed'
config.ckpt_dir = root_dir + '/ckpt/'
config.logs_dir = root_dir + '/logs/'

# ensure directories are setup
prepare_dirs(config)

config.print_values()

valid_trials : 320
test_trials : 400
way : 20
num_train : 45000
batch_size : 64
num_workers : 6
shuffle : True
augment : True
is_train : True
get_embedding : False
n_embeddings : 1000
alphabet : Mongolian
epochs : 100
init_momentum : 0.5
lr_patience : 1
train_patience : 20
use_batch_norm : True
flush : False
num_model : 4
use_gpu : False
best : True
random_seed : 1
data_dir : ../../01_data_preparation/omniglot/data/changed
ckpt_dir : C:\Users\bruno.silva\Desktop\final-project-deeplearning\02_training\siamese_networks/ckpt/
logs_dir : C:\Users\bruno.silva\Desktop\final-project-deeplearning\02_training\siamese_networks/logs/
resume : False


In [9]:
# create Omniglot data loaders
torch.manual_seed(config.random_seed)
kwargs = {}
if config.use_gpu:
    torch.cuda.manual_seed(config.random_seed)
    kwargs = {'num_workers': 1, 'pin_memory': True}
if config.is_train:
    data_loader = get_train_valid_loader(
        config.data_dir, config.batch_size,
        config.num_train, config.augment,
        config.way, config.valid_trials,
        config.shuffle, config.random_seed,
        **kwargs
    )
else:
    data_loader = get_test_loader(
        config.data_dir, config.way,
        config.test_trials, config.random_seed,
        **kwargs
    )

# sample 3 layer wise hyperparams if first time training
if config.is_train and not config.resume:
    print("[*] Sampling layer hyperparameters.")

    layer_hyperparams = {
        'layer_init_lrs': [],
        'layer_end_momentums': [],
        'layer_l2_regs': []
    }
    for i in range(6):
        # sample
        lr = random.uniform(1e-4, 1e-1)
        mom = random.uniform(0, 1)
        reg = random.uniform(0, 0.1)

        # store
        layer_hyperparams['layer_init_lrs'].append(lr)
        layer_hyperparams['layer_end_momentums'].append(mom)
        layer_hyperparams['layer_l2_regs'].append(reg)
    try:
        save_config(config, layer_hyperparams, override=True)
    except ValueError:
        print(
            "[!] Samples already exist. Either change the model number,",
            "or delete the json file and rerun.",
            sep=' '
        )
        raise
# else load it from config file
else:
    try:
        layer_hyperparams = load_config(config)
    except FileNotFoundError:
        print("[!] No previously saved config. Set resume to False.")
        raise

[*] Sampling layer hyperparameters.
[*] Model Checkpoint Dir: C:\Users\bruno.silva\Desktop\final-project-deeplearning\02_training\siamese_networks/ckpt/exp_4
[*] Param Path: C:\Users\bruno.silva\Desktop\final-project-deeplearning\02_training\siamese_networks/ckpt/exp_4\params.json


In [ ]:
trainer = Trainer(config, data_loader, layer_hyperparams)

trainer.train()

  0%|          | 0/45000 [00:00<?, ?it/s]

[*] Number of model parameters: 1,256,945

[*] Train on 45000 sample pairs, validate on 320 trials

Epoch: 1/100


0.2s - loss: 0.695: 100%|██████████| 45000/45000 [12:39<00:00, 59.24it/s]
C:\Users\bruno.silva\Desktop\final-project-deeplearning\02_training\siamese_networks\trainer.py:258: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)
  0%|          | 0/45000 [00:00<?, ?it/s]

train loss: 0.568 - val acc: 24.375 [*]

Epoch: 2/100


  0%|          | 0/45000 [00:00<?, ?it/s]

...

  0%|          | 0/45000 [00:00<?, ?it/s]

train loss: 0.084 - val acc: 87.188

Epoch: 48/100


  0%|          | 0/45000 [00:00<?, ?it/s]

train loss: 0.084 - val acc: 86.875

Epoch: 49/100


3.6s - loss: 0.059:  79%|███████▉  | 35456/45000 [35:32<08:35, 18.51it/s]